<a href="https://colab.research.google.com/github/a-nt-1/jupyter2/blob/main/Datos_externos_archivos_locales%2C_Drive%2C_Hojas_de_c%C3%A1lculo_y_Cloud_Storage_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Este cuaderno contiene recetas para cargar y guardar datos procedentes de fuentes externas.

# Sistema local de archivos

## Subir archivos desde tu sistema local de archivos

El método <code>files.upload</code> devolverá un diccionario de los archivos que se han subido.
Al diccionario se le asignará el nombre del archivo como clave y los valores serán los datos que se han subido.

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

KeyboardInterrupt: 

## Descargar archivos en tu sistema local de archivos

El método <code>files.download</code> invocará al navegador para que descargue el archivo en tu ordenador local.


In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# Descargar el fichero de la URL
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQALTRaLDFfhXOAQmeONPqmFKm9yOiQ4W97rhWgR41BZ7czFsjK5YktD6fnETKHGB9YUnyQ4XBSbhZx/pub?output=xlsx"
response = requests.get(url)
file = BytesIO(response.content)

# Leer el fichero en un DataFrame de pandas
df = pd.read_excel(file)
df

df.rename(columns={"COMB. GANADORA":"N1", "Unnamed: 2":"N2", "Unnamed: 3":"N3", "Unnamed: 4":"N4", "Unnamed: 5":"N5", "Unnamed: 6":"N6"})

#resto = empleados.loc[empleados['Nombre'] != 'Juan']    # alternativa 1
fila_ealarma = df.loc[df['FECHA']=='Periodo sin sorteos por las medidas del estado de alarma']
fila_ealarma


# agrega indice de fechas
#df['FECHA'] = pd.to_datetime(df['FECHA'])
#df.set_index('FECHA', inplace=True)


,FECHA,COMB. GANADORA,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,COMP.,R.


In [ ]:

# Preprocesar los datos
# Asumiendo que la columna de temperaturas se llama 'Temperature'
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Normalizar los datos
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Convertir los datos en una estructura de secuencias para LSTM
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 30  # Número de días para usar como secuencia
X, y = create_sequences(df_scaled, SEQ_LENGTH)

# Dividir los datos en conjuntos de entrenamiento y prueba
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Construir la red neuronal LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(X.shape[2]))

model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')

# Hacer predicciones
y_pred = model.predict(X_test)

# Desnormalizar los datos
y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

# Graficar los resultados
plt.figure(figsize=(14, 5))
plt.plot(y_test_inv, color='blue', label='Actual Temperature')
plt.plot(y_pred_inv, color='red', label='Predicted Temperature')
plt.title('Temperature Prediction')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()
plt.show()


In [ ]:
##*Seccion de código, no modificar*##

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
import requests
from io import BytesIO

# Descargar el fichero de la URL
url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQALTRaLDFfhXOAQmeONPqmFKm9yOiQ4W97rhWgR41BZ7czFsjK5YktD6fnETKHGB9YUnyQ4XBSbhZx/pub?output=xlsx"
response = requests.get(url)
file = BytesIO(response.content)

# Leer el fichero en un DataFrame de pandas
df = pd.read_excel(file)

# Preprocesar los datos
# Asumiendo que la columna de temperaturas se llama 'Temperature'
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

# Normalizar los datos
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Convertir los datos en una estructura de secuencias para LSTM
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 30  # Número de días para usar como secuencia
X, y = create_sequences(df_scaled, SEQ_LENGTH)

# Dividir los datos en conjuntos de entrenamiento y prueba
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Construir la red neuronal LSTM
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, X.shape[2])))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(X.shape[2]))

model.compile(optimizer='adam', loss='mse')

# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

# Evaluar el modelo
loss = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')

# Hacer predicciones
y_pred = model.predict(X_test)

# Desnormalizar los datos
y_test_inv = scaler.inverse_transform(y_test)
y_pred_inv = scaler.inverse_transform(y_pred)

# Graficar los resultados
plt.figure(figsize=(14, 5))
plt.plot(y_test_inv, color='blue', label='Actual Temperature')
plt.plot(y_pred_inv, color='red', label='Predicted Temperature')
plt.title('Temperature Prediction')
plt.xlabel('Time')
plt.ylabel('Temperature')
plt.legend()
plt.show()


# Google Drive

Puedes acceder a los archivos alojados en Drive de diversas maneras, como las siguientes:
- Montando tu Google Drive en la máquina virtual del entorno de ejecución
- Usando un envoltorio en la API, como <a href="https://docs.iterative.ai/PyDrive2/">PyDrive2</a>
- Usando la <a href="https://developers.google.com/drive/v3/web/about-sdk">API REST nativa</a>



A continuación se muestran ejemplos de cada método.

## Activar Google Drive de forma local

En los ejemplos siguientes se muestra cómo montar Google Drive en tu entorno de ejecución con un código de autorización y cómo puedes escribir y leer archivos en ese entorno. Cuando se haya ejecutado, podrás ver el nuevo archivo &#40;<code>foo.txt</code>&#41; en <a href="https://drive.google.com/">https://drive.google.com/</a>.

De esta manera, solo se permite la lectura, escritura y transferencia de archivos. Para modificar de forma programática las opciones para compartir u otros metadatos, utiliza una de las opciones que se indican a continuación.

<strong>Nota:</strong> Si un usuario utiliza el botón Montar Drive en el explorador de archivos, no hará falta ningún código de autenticación en los cuadernos que no haya editado nadie más.

In [ ]:
from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt

Hello Google Drive!

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')

All changes made in this colab session should now be visible in Drive.


## PyDrive2

En los ejemplos de abajo, se muestra cómo autenticarse y cómo subir o descargar archivos usando PyDrive2. Puedes consultar más ejemplos en la <a href="https://docs.iterative.ai/PyDrive2/">documentación de PyDrive2</a>.

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Autentícate y crea el cliente de PyDrive2.


In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Crea y sube un archivo de texto.


In [ ]:
uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
uploaded.SetContentString('Sample upload file content')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 14vDAdqp7BSCQnoougmgylBexIr2AQx2T


Carga un archivo por su ID e imprime su contenido.


In [ ]:
downloaded = drive.CreateFile({'id': uploaded.get('id')})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

Downloaded content "Sample upload file content"


## API REST de Drive

Para usar la API de Drive, primero debemos autenticarnos y crear un cliente de la API.


In [ ]:
from google.colab import auth
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

Con este cliente, podemos usar cualquiera de las funciones descritas en la <a href="https://developers.google.com/drive/v3/reference/">documentación de referencia de la API de Google Drive</a>. A continuación se muestran algunos ejemplos.


### Crear un nuevo archivo de Drive con datos de Python

En primer lugar, crea un archivo local que vamos a subir más adelante.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Usa el método <a href="https://developers.google.com/drive/v3/reference/files/create"><code>files.create</code></a> para subir el archivo. Para obtener más información sobre cómo subir archivos, consulta la <a href="https://developers.google.com/drive/v3/web/manage-uploads">documentación para desarrolladores</a>.

In [ ]:
from googleapiclient.http import MediaFileUpload

file_metadata = {
  'name': 'Sample file',
  'mimeType': 'text/plain'
}
media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)
created = drive_service.files().create(body=file_metadata,
                                       media_body=media,
                                       fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1Cw9CqiyU6zbXFD9ViPZu_3yX-sYF4W17


Tras ejecutar la celda anterior, aparecerá un nuevo archivo llamado "Sample file" en <a href="https://drive.google.com/">https://drive.google.com/</a>.

### Descargar datos de un archivo de Drive a Python

Descarga el archivo que hemos subido previamente.

In [ ]:
#file_id= created.get('id')

file_id= '2PACX-1vQALTRaLDFfhXOAQmeONPqmFKm9yOiQ4W97rhWgR41BZ7czFsjK5YktD6fnETKHGB9YUnyQ4XBSbhZx'
#https://docs.google.com/spreadsheets/d/e/2PACX-1vQALTRaLDFfhXOAQmeONPqmFKm9yOiQ4W97rhWgR41BZ7czFsjK5YktD6fnETKHGB9YUnyQ4XBSbhZx/pub?output=xlsx
import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, done = downloader.next_chunk()

downloaded.seek(0)
print('Downloaded file contents are: {}'.format(downloaded.read()))

NameError: name 'drive_service' is not defined

Para descargar otro archivo, asigna a <code>file&#95;id</code> el ID de ese archivo, que tendrá un formato similar a este: "1uBtlaggVyWshwcyP6kEI-y&#95;W3P8D26sz".

# Hojas de cálculo de Google

En los ejemplos siguientes, se usa la biblioteca de código abierto <a href="https://github.com/burnash/gspread"><code>gspread</code></a> para interactuar con Hojas de cálculo de Google.

Importa la biblioteca, autentícate y crea la interfaz para interactuar con Hojas de cálculo.

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

A continuación, se muestra un pequeño conjunto de ejemplos de <code>gspread</code>. Puedes consultar más ejemplos en <a href="https://github.com/burnash/gspread#more-examples">la página de GitHub de <code>gspread</code></a>.

## Crear una nueva hoja de cálculo con datos de Python

In [ ]:
sh = gc.create('My cool spreadsheet')

Tras ejecutar la celda anterior, se mostrará una nueva hoja de cálculo llamada "My cool spreadsheet" en <a href="https://sheets.google.com/">https://sheets.google.com</a>.

Abre la nueva hoja de cálculo y añade datos aleatorios.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

cell_list = worksheet.range('A1:C2')

import random
for cell in cell_list:
  cell.value = random.randint(1, 10)

worksheet.update_cells(cell_list)

{'spreadsheetId': '1dsQeN0YzXuM387l_CuyEbsYzL2ew9TJFzR-E-RQnwxs',
 'updatedCells': 6,
 'updatedColumns': 3,
 'updatedRange': 'Sheet1!A1:C2',
 'updatedRows': 2}

## Descargar los datos de una hoja de cálculo en Python como Pandas DataFrame

Vuelve a leer los datos aleatorios que hemos insertado anteriormente y convierte el resultado a <a href="https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html">Pandas DataFrame</a>.

In [ ]:
worksheet = gc.open('My cool spreadsheet').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

import pandas as pd
pd.DataFrame.from_records(rows)

[['6', '3', '4'], ['7', '2', '1']]


,0,1,2
0,6,3,4
1,7,2,1


# Google Cloud Storage &#40;GCS&#41;

Para poder usar Colaboratory con GCS, tendrás que crear un <a href="https://cloud.google.com/storage/docs/projects">proyecto de Google Cloud</a> o utilizar uno ya creado.

A continuación, especifica el ID del proyecto:

In [ ]:
project_id = 'Your_project_ID_here'

Los archivos de GCS se almacenan en <a href="https://cloud.google.com/storage/docs/buckets">segmentos</a>.

Los segmentos deben tener un nombre global único, por lo que debemos crear uno.

In [ ]:
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

Para poder acceder a GCS, debemos autenticarnos.

In [ ]:
from google.colab import auth
auth.authenticate_user()

Se puede acceder a GCS con la utilidad de línea de comandos <code>gsutil</code> o con la API de Python nativa.

## `gsutil`

En primer lugar, configuraremos <code>gsutil</code> para usar el proyecto que hemos especificado previamente con <code>gcloud</code>.

In [ ]:
!gcloud config set project {project_id}

Updated property [core/project].


Crea un archivo local que vamos a subir más adelante.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Crea el segmento donde subiremos el archivo más adelante &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/mb">documentación</a>&#41;.

In [ ]:
!gsutil mb gs://{bucket_name}

Creating gs://colab-sample-bucket-44971372-baaf-11e7-ae30-0242ac110002/...


Copia el archivo a nuestro nuevo segmento &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cp">documentación</a>&#41;.

In [ ]:
!gsutil cp /tmp/to_upload.txt gs://{bucket_name}/

Copying file:///tmp/to_upload.txt [Content-Type=text/plain]...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       


Vuelca el contenido del archivo recién copiado para asegurarte de que todo funciona correctamente &#40;<a href="https://cloud.google.com/storage/docs/gsutil/commands/cat">documentación</a>&#41;.


In [ ]:
!gsutil cat gs://{bucket_name}/to_upload.txt

my sample file

In [ ]:
# @markdown Cuando la subida haya finalizado, los datos aparecerán en el explorador de datos almacenados de la consola de Cloud de tu proyecto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Por último, vamos a descargar el archivo que acabamos de subir en el ejemplo anterior. Solo tienes que invertir el orden del comando <code>gsutil cp</code>.

In [ ]:
!gsutil cp gs://{bucket_name}/to_upload.txt /tmp/gsutil_download.txt

# Imprime el resultado para comprobar que la transferencia se ha realizado correctamente.
!cat /tmp/gsutil_download.txt

Copying gs://colab-sample-bucket483f20dc-baaf-11e7-ae30-0242ac110002/to_upload.txt...
/ [1 files][   14.0 B/   14.0 B]                                                
Operation completed over 1 objects/14.0 B.                                       
my sample file

## API de Python

Estos fragmentos proceden de <a href="https://github.com/GoogleCloudPlatform/storage-file-transfer-json-python/blob/master/chunked_transfer.py">un ejemplo más extenso</a> que muestra otros usos de la API.

En primer lugar, creamos el cliente del servicio.

In [ ]:
from googleapiclient.discovery import build
gcs_service = build('storage', 'v1')

Crea un archivo local que vamos a subir más adelante.

In [ ]:
with open('/tmp/to_upload.txt', 'w') as f:
  f.write('my sample file')

print('/tmp/to_upload.txt contains:')
!cat /tmp/to_upload.txt

/tmp/to_upload.txt contains:
my sample file

Crea un segmento en el proyecto especificado anteriormente.

In [ ]:
# Asigna al segmento un nombre globalmente único que sea distinto al del ejemplo de gsutil anterior.
import uuid
bucket_name = 'colab-sample-bucket-' + str(uuid.uuid1())

body = {
  'name': bucket_name,
  # For a full list of locations, see:
  # https://cloud.google.com/storage/docs/bucket-locations
  'location': 'us',
}
gcs_service.buckets().insert(project=project_id, body=body).execute()
print('Done')

Done


Sube el fichero al segmento que acabamos de crear.

In [ ]:
from googleapiclient.http import MediaFileUpload

media = MediaFileUpload('/tmp/to_upload.txt',
                        mimetype='text/plain',
                        resumable=True)

request = gcs_service.objects().insert(bucket=bucket_name,
                                       name='to_upload.txt',
                                       media_body=media)

response = None
while response is None:
  # _ is a placeholder for a progress object that we ignore.
  # (Our file is small, so we skip reporting progress.)
  _, response = request.next_chunk()

print('Upload complete')

Upload complete


In [ ]:
# @markdown Cuando la subida haya finalizado, los datos aparecerán en el explorador de datos almacenados de la consola de Cloud de tu proyecto:
print('https://console.cloud.google.com/storage/browser?project=' + project_id)

https://console.cloud.google.com/storage/browser?project=Your_project_ID_here


Descarga el archivo que acabamos de subir.

In [ ]:
from apiclient.http import MediaIoBaseDownload

with open('/tmp/downloaded_from_gcs.txt', 'wb') as f:
  request = gcs_service.objects().get_media(bucket=bucket_name,
                                            object='to_upload.txt')
  media = MediaIoBaseDownload(f, request)

  done = False
  while not done:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    _, done = media.next_chunk()

print('Download complete')

Download complete


Examina el archivo descargado.


In [ ]:
!cat /tmp/downloaded_from_gcs.txt

my sample file